# PyGrid: Remote Inference (Data Owner)

<img src="../../../docs/img/pygrid_logo.png" align="center"/>

Evaluate custom models using private datasets without having access to them is a powerfull resource that can change the way we interact with data during a machine learning workflow, PySyft and PyGrid offers the ability to run inferences remotely by using a variety of technologies and applications.

In this notebook series, we'll be covering all the nuances of this process, showing how to send private datasets _(as data owner)_, and how to perform remote computation using private environments _(as data scientist)_.<br><br>The main goal of these notebooks is to explore different techniques and technologies that can turn **private data _"accessible"_ while also empower data owners with total control of their own data.**

**NOTE**: _This notebook was designed to be executed in pair with the [\[Data Scientist\] PyGrid Remote Inference](http://github.com/OpenMined). In order to reproduce it properly, follow the checkpoints and instructions described in the next sections._

### Overview

- [**Initial setup**](#initial-setup)
- [**Create a compliance officer account**](#create-a-compliance-officer-account)
- [**Hosting datasets**](#hosting-datasets)
- [**Data access triage**](#data-access-request-triage)

## Initial Setup
<p>The first thing to do after deploying a domain, is to set its initial configuration. Here we can define some values and flags that will change how the domain will behave. We also use this step to create the owner account, which will have full control and access to the platform. As long as this step is not executed, all other endpoints/features will be blocked.</p>

### Importing libs

In [1]:
from syft.grid.client.client import connect # Method used to connect with the domain.
from syft.grid.client.grid_connection import GridHTTPConnection # Protocol used to talk with the domain

import syft as sy
import torch as th

# Set logging level
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
# 1 - Since we still don't have any account registered,
# we can connect with the domain without credentials.
domain_client = connect(
    url="http://localhost:5000", # Domain Address
    conn_type=GridHTTPConnection,) # HTTP Connection Protocol

# 2 - Set the domain initial configuration
# - Owner's email/password
# - Domain Name used to identify this owner.
# - Random token generated during deployment
domain_client.setup(
    email="owner@myorg.com",
    password="owerpwd",
    domain_name="OpenMined Domain",
    token="9G9MJ06OQH"
)


# 3 - Now, we have an account registered!
# Let's connect with the domain as using owner credentials.
domain = connect(
    url="http://localhost:5000", # Domain Address
    credentials={"email":"owner@myorg.com", "password":"owerpwd"},
    conn_type= GridHTTPConnection,) # HTTP Connection Protocol

INFO:root:Running initial setup!


_PS : For didatic purposes, we have reduced the number of parameters used in the "setup" process. But the domain also accepts a set of optional parameters. If you want to know more about the initial setup process, it's highly recommended to read our roadmap._

## Hosting Datasets

As a domain owner, we're also able to upload new datasets to the domain. We can do it by using two different approaches:

- **Tensor Data Structures**: Using the torch tensor structures and the default PySyft API
- **Structured and compressed CSV folder** : We can upload a _.tar.gz_ file  by using the PyGrid API.

We'll be covering the second one, sending a compressed file containing a structured folder with different CSV files + metadata information about the dataset _(manifest, description and tags)_.

In [3]:
dataset = domain.datasets.create('diabetes-dataset.tar.gz')
dataset.pandas

,name,id,tags,dtype,shape
0,diabetes-data/data_02.csv,3cb1b048-02d7-40b2-9e39-7834fd98fa96,"[#diabetes, #pima-indians-database, #data_02.csv]",Tensor,"(200, 9)"
1,diabetes-data/data_06.csv,0701de80-8913-469e-b9af-45ad0046d292,"[#diabetes, #pima-indians-database, #data_06.csv]",Tensor,"(200, 9)"
2,diabetes-data/data_10.csv,ce9955d0-89d5-40bc-99e1-2f6dbc13e3da,"[#diabetes, #pima-indians-database, #data_10.csv]",Tensor,"(200, 9)"
3,diabetes-data/data_03.csv,3973793e-3ff6-431a-9789-793e259c0ddd,"[#diabetes, #pima-indians-database, #data_03.csv]",Tensor,"(200, 9)"
4,diabetes-data/data_04.csv,b0bbc51e-fc61-4c93-9335-dd4f39a2baa2,"[#diabetes, #pima-indians-database, #data_04.csv]",Tensor,"(200, 9)"
5,diabetes-data/new_data.csv,c4bd6609-5ed9-4432-bc1b-390f11270a9e,"[#diabetes, #pima-indians-database, #new_data....",Tensor,"(2000, 9)"
6,diabetes-data/data_07.csv,3508dfc3-941c-42e7-a2f1-0ddee84b3bb2,"[#diabetes, #pima-indians-database, #data_07.csv]",Tensor,"(200, 9)"
7,diabetes-data/data_08.csv,b35cb625-00ef-494b-a318-21115ce2c695,"[#diabetes, #pima-indians-database, #data_08.csv]",Tensor,"(200, 9)"
8,diabetes-data/data_09.csv,213c1e8e-2520-4d82-92e5-d9a4ff202021,"[#diabetes, #pima-indians-database, #data_09.csv]",Tensor,"(200, 9)"
9,diabetes-data/data_05.csv,37fc6be6-157e-4c7e-987c-03df0447d35a,"[#diabetes, #pima-indians-database, #data_05.csv]",Tensor,"(200, 9)"


Now, the dataset is available to be used remotely by a data scientist.

## Create a compliance officer account
By default, PyGrid initializes with 4 pre-set roles that can be attached to the users. These roles are used to define what a user is allowed or not allowed to do when logged-in. You can check its permissions by executing this code snippet.

### Roles

In [4]:
domain.roles.all(pandas=True)

,id,name,can_triage_requests,can_edit_settings,can_create_users,can_create_groups,can_edit_roles,can_manage_infrastructure,can_upload_data
0,1,User,False,False,False,False,False,False,False
1,2,Compliance Officer,True,False,False,False,False,False,False
2,3,Administrator,True,True,True,True,False,False,True
3,4,Owner,True,True,True,True,True,True,True


These roles are not static _(excluding owner role)_ and can be updated or deleted if necessary. You can also create new ones if needed.

Using owner permissions, you can also check and change the roles used by each user registered in the domain.

In [5]:
domain.users.all(pandas=True)

,id,email,private_key,verify_key,role,groups
0,1,owner@myorg.com,4cc3337abc17833f230e9ad40a3a497b4db372b6583334...,5a4b76384dd0708608a6bb7985f823cb8c319f8dcb3e25...,4,[]


### User Creation

PyGrid allow people to create new accounts by themselves, therefore, these accounts will always be  attached to the User role _(without any management permissions)_. Considering the default roles, only the Owner and Administrator accounts would be able to create new accounts attached with different permissions.<br><br>
For our use case, we'll create a **compliance officer** account. This role is commonly assigned for users who will **evaluate data access requests**, approving or rejecting them. Since we're logged-in as a owner, we have all the requirements.

In [6]:
domain.users.create(
    email="compliance@myorg.com",
    password="pwd123",
    role="Compliance Officer"
)

INFO:root:User created successfully!


In [7]:
domain.users.all(pandas=True)

,id,email,private_key,verify_key,role,groups
0,1,owner@myorg.com,4cc3337abc17833f230e9ad40a3a497b4db372b6583334...,5a4b76384dd0708608a6bb7985f823cb8c319f8dcb3e25...,4,[]
1,2,compliance@myorg.com,5b1e31e3e3c2a2b897eb3ebf81f9e674a7b1471ef0f116...,0361d9063babfdc08a8e3d78eeee812c3312e3ccc4291f...,2,[]


### Login as a Compliance Officer
To be accurate to this use case, we'll log-in as compliance officer.<br><br>
**From now on, we'll use only compliance officer credentials to go through this use case.**

In [8]:
cp_officer = connect(
    url="http://localhost:5000", # Domain Address
    credentials={"email":"compliance@myorg.com", "password":"pwd123"},
    conn_type= GridHTTPConnection,) # HTTP Connection Protocol

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint : Now STOP and run the Data Scientist notebook until the next checkpoint.

## Data access triage

Empowered by the data access management, the compliance officer the evaluate each request accepting or denying them.

In [9]:
cp_officer.requests.pandas

,Requested Object's tags,Reason,Request ID,Requested Object's ID,Requested Object's type
0,"[#diabetes, #pima-indians-database, #data_02.c...",I'd like to have access to my accuracy result!,<UID: 5ebb973b356644dda0f16ff2d256de2a>,<UID: 0cbd040781a04559ba40e983723d8d2c>,<class 'syft.lib.python.Float'>


In [10]:
cp_officer.requests[0].accept()

## Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft and SyMPC on GitHub
The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

* [Star PySyft](https://github.com/OpenMined/PySyft)
* [Star PyGrid](https://github.com/OpenMined/PyGrid)

### Join our Slack!
The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at http://slack.openmined.org

### Join a Code Project!
The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

* [PySyft Good First Issue Tickets](https://github.com/OpenMined/PySyft/labels/Good%20first%20issue%20%3Amortar_board%3A)
* [PyGrid Good First Issue Tickets](https://github.com/OpenMined/PyGrid/labels/good%20first%20issue)

### Donate
If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

* [OpenMined's Open Collective Page](https://opencollective.com/openmined)